In [1]:
import pandas as pd

In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
class CNN_Model(nn.Module):
    def __init__(self, input_channels, classes):
        super(CNN_Model, self).__init__()

        
        self.conv1 = nn.Conv2d(in_channels=input_channels,out_channels= 20, kernel_size=(5,5))
        self.relu1= nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50,kernel_size=(5, 5))
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.fc1 = nn.Linear(in_features=800,out_features=500)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=500, out_features=classes)
        self.logSoftmax = nn.LogSoftmax(dim=1)

    
    def forward(self, x):

        x=self.conv1(x)
        x=self.relu1(x)
        x=self.pool1(x)
        
        x=self.conv2(x)
        x=self.relu2(x)
        x=self.pool2(x)
        
        # x=self.conv3(x)
        # x=self.relu3(x)
        # x=self.pool3(x)

        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.relu4(x)

        x = self.fc2(x)
        output = self.logSoftmax(x)

        return output

In [5]:
init_lr= 0.01
batch_size = 64
epochs = 5

train_split = 0.75
val_split = 1- train_split

In [6]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torchvision

In [8]:
print("[INFO] loading the KMNIST dataset...")
train_data = torchvision.datasets.MNIST(root="data", train= True, download = True, transform= torchvision.transforms.ToTensor())

test_data = torchvision.datasets.MNIST(root="data", train= False, download = True, transform= torchvision.transforms.ToTensor())

print("[INFO] generating the train/validation split...")
number_of_train_samples = int(len(train_data)*train_split)

number_of_value_samples = int(len(train_data)*val_split)

train_data, val_data = torch.utils.data.random_split(train_data, [number_of_train_samples, number_of_value_samples], generator= torch.manual_seed(100))

[INFO] loading the KMNIST dataset...
[INFO] generating the train/validation split...


In [9]:
# initialize the train, validation, and test data loaders
trainDataLoader = torch.utils.data.DataLoader(train_data, shuffle=True,
	batch_size=batch_size)
valDataLoader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
testDataLoader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
# calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // batch_size
valSteps = len(valDataLoader.dataset) // batch_size

In [10]:
# initialize the LeNet model
import time

import numpy as np
from sklearn.metrics import classification_report

print("[INFO] initializing the LeNet model...")
model = CNN_Model(
	input_channels=1,
	classes=len(train_data.dataset.classes))
# initialize our optimizer and loss function
opt = torch.optim.Adam(model.parameters(), lr=init_lr)
lossFn = nn.NLLLoss()
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

# loop over our epochs
for e in range(0, epochs):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (x, y) in trainDataLoader:
		# send the input to the device
		# (x, y) = (x.to(device), y.to(device))
		# perform a forward pass and calculate the training loss
		pred = model(x)
		loss = lossFn(pred, y)
		# zero out the gradients, perform the backpropagation step,
		# and update the weights
		opt.zero_grad()
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()
		
	with torch.no_grad():
		# set the model in evaluation mode
		model.eval()
		# loop over the validation set
		for (x, y) in valDataLoader:
			# send the input to the device
			# (x, y) = (x.to(device), y.to(device))
			# make the predictions and calculate the validation loss
			pred = model(x)
			totalValLoss += lossFn(pred, y)
			# calculate the number of correct predictions
			valCorrect += (pred.argmax(1) == y).type(
				torch.float).sum().item()
			
	# calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgValLoss = totalValLoss / valSteps
	# calculate the training and validation accuracy
	trainCorrect = trainCorrect / len(trainDataLoader.dataset)
	valCorrect = valCorrect / len(valDataLoader.dataset)
	# update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["train_acc"].append(trainCorrect)
	H["val_loss"].append(avgValLoss.cpu().detach().numpy())
	H["val_acc"].append(valCorrect)
	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, epochs))
	print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
		avgTrainLoss, trainCorrect))
	print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
		avgValLoss, valCorrect))
	
# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))
# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
with torch.no_grad():
	# set the model in evaluation mode
	model.eval()
	
	# initialize a list to store our predictions
	preds = []
	# loop over the test set
	for (x, y) in testDataLoader:
		# send the input to the device
		# x = x.to(device)
		# make the predictions and add them to the list
		pred = model(x)
		preds.extend(pred.argmax(axis=1).cpu().numpy())
# generate a classification report
print(classification_report(test_data.targets.cpu().numpy(),
	np.array(preds), target_names=test_data.classes))

[INFO] initializing the LeNet model...
[INFO] training the network...
[INFO] EPOCH: 1/5
Train loss: 0.408213, Train accuracy: 0.8696
Val loss: 0.226640, Val accuracy: 0.9275

[INFO] EPOCH: 2/5
Train loss: 0.200842, Train accuracy: 0.9376
Val loss: 0.154778, Val accuracy: 0.9503

[INFO] EPOCH: 3/5
Train loss: 0.170080, Train accuracy: 0.9487
Val loss: 0.171008, Val accuracy: 0.9474

[INFO] EPOCH: 4/5
Train loss: 0.158390, Train accuracy: 0.9511
Val loss: 0.173160, Val accuracy: 0.9469

[INFO] EPOCH: 5/5
Train loss: 0.157425, Train accuracy: 0.9523
Val loss: 0.150531, Val accuracy: 0.9549

[INFO] total time taken to train the model: 155.97s
[INFO] evaluating network...
              precision    recall  f1-score   support

    0 - zero       0.95      0.98      0.97       980
     1 - one       0.99      0.98      0.99      1135
     2 - two       0.97      0.95      0.96      1032
   3 - three       0.96      0.97      0.97      1010
    4 - four       0.97      0.94      0.96       982

In [11]:
torch.save(model, "MNIST_model.pth")